In [136]:
"""
Riddler Classic 2022-05-13
https://fivethirtyeight.com/features/its-elementary-my-dear-riddler/
"""

import numpy as np
from itertools import product
from collections import Counter

"""
This game can be represented as a 5-state Markov model. 
The five states are the initial roll, defeat, roll 3 dice, roll 2 dice, victory.
Defeat and victory are absorbing states.

This function returns the number of outcomes for each count of distinct dice remaining.
If there is 1 distinct dice remaining this is a loss and if there are 4 this is a win.
If there are 2 pairs of duplicates this is a loss (we code this as one dice remaining for our Markov model)
"""
def outcomes(num_dice):
    
    # Generate permutations of rolling N dice
    perms = product(range(1,5),repeat=num_dice)
    
    # Consider all 4 dice and add back in the ones that weren't rolled
    if num_dice == 4:
        perms_amended = perms
    elif num_dice == 3:
        perms_amended = [list(i) + [1] for i in perms] # Assume 1 is the not the duplicate
    elif num_dice == 2:
        perms_amended = [list(i) + [1,2] for i in perms] # Assume 1,2 is not the duplicate
    
    # Return number of distinct values, which determines the probability of reaching each state
    # Overwriting the values in the (2,2) case - not a pretty way to do this but it works as a hack.
    ct_distinct_vals = [1 if max(Counter(i).values()) == 2 and min(Counter(i).values()) == 2 else len(Counter(i).keys()) for i in perms_amended]  
    
    # For ease of creating our Markov transition matrix, we rename our dictionary keys so they refer to the right order
    # Because the absorbing states in our matrix have to be last
    ct = Counter(ct_distinct_vals)
    if 1 in ct.keys():
        ct[5] = ct.pop(1)
    return ct

outcomes_arr = []
for n in range(4,1,-1): # Go in reverse to make matrix construction easier
    outcomes_arr.append(outcomes(num_dice=n))

# Create a Markov transition matrix for our 5-state model
# Now in order of initial roll, roll 3, roll 2, victory, defeat.
states = 5
M = np.zeros((states,states))
for y in range(states-2):
    for x in range(1,states):
        M[y,x] = outcomes_arr[y][x+1]/sum(outcomes_arr[y].values())

# Absorbing states
M[3,3], M[4,4] = 1,1

# Matrix algebra
# Method described in https://github.com/khgiddon/misc/blob/main/riddler_2020_10_23_notebook.ipynb
I = np.eye(len(M) - 2)  
Q = M[:-2, :-2]
R = M[:-2,-2:]
B = np.matmul(np.linalg.inv(I - Q), R)

# Probability of winning for starting state
print(B[0,0])

# 0.45


0.45000000000000007
